In [1]:
import snappy
from snappy import ProductIO
import datetime
import sys
import os
import gc
import calibrate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from snappy import *
subsetop = snappy.jpy.get_type('org.esa.snap.core.gpf.common.SubsetOp')
wktreader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
pixelpos = snappy.jpy.get_type('org.esa.snap.core.datamodel.PixelPos')
geopos = snappy.jpy.get_type('org.esa.snap.core.datamodel.GeoPos')

In [2]:
def subset(files,  w, bandlist1):
    wkt = wktreader()
    geometries = wkt.read(w)
    subop = subsetop()
    subop.setParameterDefaultValues()
    subop.setSourceProduct(files)
    subop.setGeoRegion(geometries)
    subop.setCopyMetadata(True)
    subop.setBandNames(bandlist1)
    try:
        dd = subop.getTargetProduct()
    except Exception:
        print('exception')
        dd=False
    print('--------success-----')
    return dd

In [14]:
file_ = ProductIO.readProduct('../ecmwf/east02.nc')

In [15]:
bandlist=['u10_time1','v10_time1']
w='POLYGON((-68 26,-62 27,-64 33,-69 34,-68 26))'

In [16]:
file_sb = subset(file_, w, bandlist)

--------success-----


In [17]:
ProductIO.writeProduct(file_sb, 'file_sb','BEAM-DIMAP')

In [15]:
band = file_sb.getBand('u10_time1')

In [16]:
band = file_.getBand('u10_time1')

In [22]:
band.getImageInfo()

In [23]:
file_g = file_.getSceneGeoCoding()

In [3]:
import pandas as pd
import numpy as np

In [6]:
sar_information = pd.read_csv('./asf-datapool-results-2018-4-4_15-34-49.csv', index_col='Granule Name')

In [24]:
sar_information.columns

Index([u'Platform', u'Sensor', u'Beam Mode', u'Beam Mode Description',
       u'Absolute Orbit', u'Relative Orbit', u'Path Number', u'Frame Number',
       u'Acquisition Date', u'Processing Date', u'Processing Level',
       u'Start Time', u'End Time', u'Center Lat', u'Center Lon',
       u'Near Start Lat', u'Near Start Lon', u'Far Start Lat',
       u'Far Start Lon', u'Near End Lat', u'Near End Lon', u'Far End Lat',
       u'Far End Lon', u'Faraday Rotation', u'Ascending or Descending?',
       u'URL', u'Size (MB)', u'Off Nadir Angle', u'Stack Size',
       u'Baseline Perp.', u'Doppler', u'Stack ID'],
      dtype='object')

In [26]:
sar_information.loc[u'S1B_IW_GRDH_1SDV_20171123T222438_20171123T222503_008415_00EE6F_204A', u'Far Start Lon']

-65.270270999999994

In [4]:
def get_wkt(file_name):
    far_start_lon = sar_information.loc[file_name, u'Far Start Lon']
    far_start_lat = sar_information.loc[file_name, u'Far Start Lat']
    far_end_lon = sar_information.loc[file_name, u'Far Start Lon']
    far_end_lat = sar_information.loc[file_name, u'Far Start Lat']
    near_start_lat = sar_information.loc[file_name, u'Near Start Lat']
    near_start_lon = sar_information.loc[file_name, u'Near Start Lon']
    near_end_lat = sar_information.loc[file_name, u'Near End Lat']
    near_end_lon = sar_information.loc[file_name, u'Near End Lon']
    w = 'POLYGON((%f %f, %f %f, %f %f, %f %f, %f %f))'%(near_start_lon, near_start_lat, near_end_lon, near_end_lat, far_end_lon, far_end_lat, far_start_lon, far_start_lat, near_start_lon, near_start_lat)
    return w


In [5]:
def get_time(file_):
    file_part_list = file_.split('_')
    time1 = file_part_list[4]
    time_part = time1.split('T')
    date = time_part[0]
    time = time_part[1]
    year = int(date[:4])
    month = int(date[4:6])
    day = int(date[6:])
    hour = int(time[:2])
    minute = int(time[2:4])
    data_list = [year, month, day, hour, minute]
    print('\n------time------')
    print(data_list)
    return data_list

'''
get bandlist
'''
def get_bandlist(file1):
    time= get_time(file1)
    date1 = datetime.datetime(2015, 10,1,0,0,0)
    date2 = datetime.datetime(time[0], time[1], time[2], time[3], time[4],0)
    dd = date2-date1
    band1 = int(np.around(dd.total_seconds()/60/60/6))+1
    band2 = int(dd.total_seconds()/60/60/6)+1
    if band1 == band2:
        band2 +=1
    bandlist=['u10_time%d'%band1, 'v10_time%d'%band1,'u10_time%d'%band2, 'v10_time%d'%band2]
    print('\n------bandlist------')
    print(bandlist)
    return bandlist


In [40]:
sar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1146 entries, 0 to 1145
Data columns (total 33 columns):
Granule Name                1146 non-null object
Platform                    1146 non-null object
Sensor                      1146 non-null object
Beam Mode                   1146 non-null object
Beam Mode Description       1146 non-null object
Absolute Orbit              1146 non-null int64
Relative Orbit              1146 non-null int64
Path Number                 1146 non-null int64
Frame Number                1146 non-null int64
Acquisition Date            1146 non-null object
Processing Date             1146 non-null object
Processing Level            1146 non-null object
Start Time                  1146 non-null object
End Time                    1146 non-null object
Center Lat                  1146 non-null float64
Center Lon                  1146 non-null float64
Near Start Lat              1146 non-null float64
Near Start Lon              1146 non-null float64
Far Start L

In [7]:
sar = sar_information.reset_index()

ImportError: No module named tqdm

In [8]:
for file_n in sar['Granule Name']:
    band = get_bandlist(file_n)
    wk = get_wkt(file_n)
    if sar_information.loc[file_n,'Center Lat'] <60 and sar_information.loc[file_n,'Center Lat']>28 and sar_information.loc[file_n,'Center Lon']>-135 and sar_information.loc[file_n,'Center Lon']<-110:
        f = ProductIO.readProduct('../ecmwf/west02.nc')
    elif sar_information.loc[file_n,'Center Lat'] <50 and sar_information.loc[file_n,'Center Lat']>10 and sar_information.loc[file_n,'Center Lon']>-84 and sar_information.loc[file_n,'Center Lon']<-48:
        f = ProductIO.readProduct('../ecmwf/east02.nc')
    sb = subset(f, wk, band)
    if sb is False:
        f.dispose()
        del sb
        del band
        del wk
        gc.collect()
        continue
    ProductIO.writeProduct(sb, '/Volumes/Yangchao/sar_ecmwf/'+sar_information.loc[file_n, u'Ascending or Descending?']+'_'+file_n, 'BEAM-DIMAP')
    f.dispose()
    sb.dispose()
    del band
    del wk
    gc.collect()


------time------
[2017, 11, 23, 22, 24]

------bandlist------
['u10_time3141', 'v10_time3141', 'u10_time3140', 'v10_time3140']
--------success-----

------time------
[2017, 11, 13, 22, 8]

------bandlist------
['u10_time3101', 'v10_time3101', 'u10_time3100', 'v10_time3100']
--------success-----

------time------
[2017, 11, 3, 10, 50]

------bandlist------
['u10_time3059', 'v10_time3059', 'u10_time3058', 'v10_time3058']
--------success-----

------time------
[2017, 7, 25, 9, 57]

------bandlist------
['u10_time2655', 'v10_time2655', 'u10_time2654', 'v10_time2654']
--------success-----

------time------
[2017, 7, 23, 22, 50]

------bandlist------
['u10_time2649', 'v10_time2649', 'u10_time2648', 'v10_time2648']
--------success-----

------time------
[2017, 7, 23, 22, 50]

------bandlist------
['u10_time2649', 'v10_time2649', 'u10_time2648', 'v10_time2648']
--------success-----

------time------
[2017, 7, 23, 22, 49]

------bandlist------
['u10_time2649', 'v10_time2649', 'u10_time2648', '

['u10_time2533', 'v10_time2533', 'u10_time2532', 'v10_time2532']
--------success-----

------time------
[2017, 6, 24, 22, 37]

------bandlist------
['u10_time2533', 'v10_time2533', 'u10_time2532', 'v10_time2532']
--------success-----

------time------
[2017, 6, 24, 10, 5]

------bandlist------
['u10_time2531', 'v10_time2531', 'u10_time2530', 'v10_time2530']
--------success-----

------time------
[2017, 6, 24, 10, 5]

------bandlist------
['u10_time2531', 'v10_time2531', 'u10_time2530', 'v10_time2530']
--------success-----

------time------
[2017, 6, 22, 22, 54]

------bandlist------
['u10_time2525', 'v10_time2525', 'u10_time2524', 'v10_time2524']
--------success-----

------time------
[2017, 6, 22, 10, 21]

------bandlist------
['u10_time2523', 'v10_time2523', 'u10_time2522', 'v10_time2522']
--------success-----

------time------
[2017, 6, 22, 10, 21]

------bandlist------
['u10_time2523', 'v10_time2523', 'u10_time2522', 'v10_time2522']
--------success-----

------time------
[2017, 6, 

------bandlist------
['u10_time2433', 'v10_time2433', 'u10_time2434', 'v10_time2434']
--------success-----

------time------
[2017, 5, 31, 2, 6]

------bandlist------
['u10_time2433', 'v10_time2433', 'u10_time2434', 'v10_time2434']
--------success-----

------time------
[2017, 5, 31, 2, 6]

------bandlist------
['u10_time2433', 'v10_time2433', 'u10_time2434', 'v10_time2434']
--------success-----

------time------
[2017, 5, 31, 2, 6]

------bandlist------
['u10_time2433', 'v10_time2433', 'u10_time2434', 'v10_time2434']
--------success-----

------time------
[2017, 5, 31, 2, 5]

------bandlist------
['u10_time2433', 'v10_time2433', 'u10_time2434', 'v10_time2434']
--------success-----

------time------
[2017, 5, 31, 2, 5]

------bandlist------
['u10_time2433', 'v10_time2433', 'u10_time2434', 'v10_time2434']
--------success-----

------time------
[2017, 5, 30, 23, 42]

------bandlist------
['u10_time2433', 'v10_time2433', 'u10_time2432', 'v10_time2432']
exception
--------success-----

----

['u10_time2417', 'v10_time2417', 'u10_time2416', 'v10_time2416']
--------success-----

------time------
[2017, 5, 26, 22, 35]

------bandlist------
['u10_time2417', 'v10_time2417', 'u10_time2416', 'v10_time2416']
--------success-----

------time------
[2017, 5, 26, 22, 34]

------bandlist------
['u10_time2417', 'v10_time2417', 'u10_time2416', 'v10_time2416']
--------success-----

------time------
[2017, 5, 26, 22, 29]

------bandlist------
['u10_time2417', 'v10_time2417', 'u10_time2416', 'v10_time2416']
--------success-----

------time------
[2017, 5, 26, 22, 28]

------bandlist------
['u10_time2417', 'v10_time2417', 'u10_time2416', 'v10_time2416']
--------success-----

------time------
[2017, 5, 26, 16, 32]

------bandlist------
['u10_time2416', 'v10_time2416', 'u10_time2415', 'v10_time2415']
exception
--------success-----

------time------
[2017, 5, 26, 12, 25]

------bandlist------
['u10_time2415', 'v10_time2415', 'u10_time2416', 'v10_time2416']
exception
--------success-----

-----

--------success-----

------time------
[2017, 5, 23, 14, 22]

------bandlist------
['u10_time2403', 'v10_time2403', 'u10_time2404', 'v10_time2404']
--------success-----

------time------
[2017, 5, 23, 14, 22]

------bandlist------
['u10_time2403', 'v10_time2403', 'u10_time2404', 'v10_time2404']
--------success-----

------time------
[2017, 5, 23, 14, 21]

------bandlist------
['u10_time2403', 'v10_time2403', 'u10_time2404', 'v10_time2404']
--------success-----

------time------
[2017, 5, 23, 11, 9]

------bandlist------
['u10_time2403', 'v10_time2403', 'u10_time2402', 'v10_time2402']
--------success-----

------time------
[2017, 5, 23, 4, 47]

------bandlist------
['u10_time2402', 'v10_time2402', 'u10_time2401', 'v10_time2401']
exception
--------success-----

------time------
[2017, 5, 23, 4, 46]

------bandlist------
['u10_time2402', 'v10_time2402', 'u10_time2401', 'v10_time2401']
exception
--------success-----

------time------
[2017, 5, 23, 4, 45]

------bandlist------
['u10_time240

...

[2017, 5, 8, 23, 23]

------bandlist------
['u10_time2345', 'v10_time2345', 'u10_time2344', 'v10_time2344']
--------success-----

------time------
[2017, 5, 8, 23, 23]

------bandlist------
['u10_time2345', 'v10_time2345', 'u10_time2344', 'v10_time2344']
--------success-----

------time------
[2017, 5, 8, 23, 20]

------bandlist------
['u10_time2345', 'v10_time2345', 'u10_time2344', 'v10_time2344']
--------success-----

------time------
[2017, 5, 8, 23, 20]

------bandlist------
['u10_time2345', 'v10_time2345', 'u10_time2344', 'v10_time2344']
--------success-----

------time------
[2017, 5, 8, 23, 19]

------bandlist------
['u10_time2345', 'v10_time2345', 'u10_time2344', 'v10_time2344']
--------success-----

------time------
[2017, 5, 8, 23, 19]

------bandlist------
['u10_time2345', 'v10_time2345', 'u10_time2344', 'v10_time2344']
--------success-----

------time------
[2017, 5, 8, 14, 1]

------bandlist------
['u10_time2343', 'v10_time2343', 'u10_time2344', 'v10_time2344']
--------suc

['u10_time2325', 'v10_time2325', 'u10_time2324', 'v10_time2324']
--------success-----

------time------
[2017, 5, 3, 22, 26]

------bandlist------
['u10_time2325', 'v10_time2325', 'u10_time2324', 'v10_time2324']
--------success-----

------time------
[2017, 5, 3, 22, 25]

------bandlist------
['u10_time2325', 'v10_time2325', 'u10_time2324', 'v10_time2324']
--------success-----

------time------
[2017, 5, 3, 22, 25]

------bandlist------
['u10_time2325', 'v10_time2325', 'u10_time2324', 'v10_time2324']
--------success-----

------time------
[2017, 5, 2, 22, 35]

------bandlist------
['u10_time2321', 'v10_time2321', 'u10_time2320', 'v10_time2320']
--------success-----

------time------
[2017, 5, 2, 22, 34]

------bandlist------
['u10_time2321', 'v10_time2321', 'u10_time2320', 'v10_time2320']
--------success-----

------time------
[2017, 5, 2, 22, 29]

------bandlist------
['u10_time2321', 'v10_time2321', 'u10_time2320', 'v10_time2320']
--------success-----

------time------
[2017, 5, 2, 2

--------success-----

------time------
[2017, 4, 25, 22, 37]

------bandlist------
['u10_time2293', 'v10_time2293', 'u10_time2292', 'v10_time2292']
--------success-----

------time------
[2017, 4, 25, 10, 48]

------bandlist------
['u10_time2291', 'v10_time2291', 'u10_time2290', 'v10_time2290']
--------success-----

------time------
[2017, 4, 25, 10, 48]

------bandlist------
['u10_time2291', 'v10_time2291', 'u10_time2290', 'v10_time2290']
--------success-----

------time------
[2017, 4, 25, 10, 5]

------bandlist------
['u10_time2291', 'v10_time2291', 'u10_time2290', 'v10_time2290']
--------success-----

------time------
[2017, 4, 25, 10, 5]

------bandlist------
['u10_time2291', 'v10_time2291', 'u10_time2290', 'v10_time2290']
--------success-----

------time------
[2017, 4, 25, 3, 51]

------bandlist------
['u10_time2290', 'v10_time2290', 'u10_time2289', 'v10_time2289']
exception
--------success-----

------time------
[2017, 4, 25, 3, 51]

------bandlist------
['u10_time2290', 'v10_t


------bandlist------
['u10_time2257', 'v10_time2257', 'u10_time2256', 'v10_time2256']
--------success-----

------time------
[2017, 4, 16, 22, 17]

------bandlist------
['u10_time2257', 'v10_time2257', 'u10_time2256', 'v10_time2256']
--------success-----

------time------
[2017, 4, 15, 22, 20]

------bandlist------
['u10_time2253', 'v10_time2253', 'u10_time2252', 'v10_time2252']
--------success-----

------time------
[2017, 4, 15, 22, 20]

------bandlist------
['u10_time2253', 'v10_time2253', 'u10_time2252', 'v10_time2252']
--------success-----

------time------
[2017, 4, 15, 9, 49]

------bandlist------
['u10_time2251', 'v10_time2251', 'u10_time2250', 'v10_time2250']
--------success-----

------time------
[2017, 4, 15, 9, 49]

------bandlist------
['u10_time2251', 'v10_time2251', 'u10_time2250', 'v10_time2250']
--------success-----

------time------
[2017, 4, 15, 3, 35]

------bandlist------
['u10_time2250', 'v10_time2250', 'u10_time2249', 'v10_time2249']
exception
--------success---

--------success-----

------time------
[2017, 3, 3, 2, 54]

------bandlist------
['u10_time2077', 'v10_time2077', 'u10_time2078', 'v10_time2078']
exception
--------success-----

------time------
[2017, 3, 3, 2, 54]

------bandlist------
['u10_time2077', 'v10_time2077', 'u10_time2078', 'v10_time2078']
exception
--------success-----

------time------
[2017, 3, 3, 2, 53]

------bandlist------
['u10_time2077', 'v10_time2077', 'u10_time2078', 'v10_time2078']
exception
--------success-----

------time------
[2017, 2, 26, 5, 11]

------bandlist------
['u10_time2058', 'v10_time2058', 'u10_time2057', 'v10_time2057']
exception
--------success-----

------time------
[2017, 2, 26, 2, 45]

------bandlist------
['u10_time2057', 'v10_time2057', 'u10_time2058', 'v10_time2058']
exception
--------success-----

------time------
[2017, 2, 24, 3, 51]

------bandlist------
['u10_time2050', 'v10_time2050', 'u10_time2049', 'v10_time2049']
exception
--------success-----

------time------
[2017, 2, 24, 3, 50]



In [53]:
f = ProductIO.readProduct('../ecmwf/east02.nc')

In [54]:
band = f.getBand('u10_time2437')

In [61]:
databox2=np.zeros(1*1)

In [62]:
band.readPixels(10,10,1,1,databox2)

array([ 1.22391962])

In [ ]:
band.readPixels(f.)

In [63]:
dd = f.getSceneGeoCoding()

In [67]:
pp = dd.getPixelPos(geopos(40, -50),None)

In [68]:
pp.x

272.5

In [69]:
pp.y

80.5

In [2]:
import numpy as np

mask out

In [2]:
file_list = []
with os.popen('cd /Volumes/Yangchao/sar_ecmwf/ \n ls *.dim') as file_lists:
    for f in file_lists:
        file_list.append(f[:-1])
        file_ecmwf = ProductIO.readProduct('/Volumes/Yangchao/sar_ecmwf/'+f[:-1])
        file_ecmwf_mask = calibrate.mask(file_ecmwf)
        ProductIO.writeProduct(file_ecmwf_mask, '/Volumes/Yangchao/sar_ecmwf_mask/'+f.split('.')[0], 'BEAM-DIMAP')
        file_ecmwf.dispose()
        file_ecmwf_mask.dispose()
        gc.collect()
#         file_ = ProductIO.readProduct('/Volumes/Yangchao/Sentient/'+'_'.join(f[:-1].split('_')[1:]).split('.')[0]+'.zip')
#         file_them_app = calibrate.thermal_app(file_)
#         file_cali = calibrate.calibrate(file_them_app)
#         file_mask = calibrate.mask(file_cali)
#         ProductIO.writeProduct(file_mask, '/Volumes/Yangchao/sentinel_t_o_c_m/'+f.split('.')[0],'BEAM-DIMAP')
#         file_.dispose()
#         file_them_app.dispose()
#         file_cali.dispose()
#         file_mask.dispose()
#         gc.collect()
np.save(file_list, 'file_list')

----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----
----mask----

AttributeError: 'list' object has no attribute 'write'

In [11]:
gc.collect()

0

In [ ]:
ndbc_information = 

In [24]:
file_ = ProductIO.readProduct('/Volumes/Yangchao/sar_ecmwf_mask/Ascending_S1A_IW_GRDH_1SDV_20170723T225031_20170723T225056_017605_01D74D_AA68.dim')

In [25]:
bandnames=file_.getBandNames()

In [26]:
band = file_.getBand(bandnames[0])

In [11]:
size = file_.getSceneRasterSize()

In [12]:
ar = np.zeros((size.height,size.width))

In [13]:
ar = band.readPixels(0,0, size.height, size.width,ar)

In [50]:
mask_g = file_.getMaskGroup()

In [28]:
image = band.getValidMaskImage()

In [33]:
data = image.getData()

In [42]:
data

java.awt.image.Raster(objectRef=0x7f89e9af7518)

In [44]:
dd = data.getPixels(0,0,1,1,np.zeros(1))

In [48]:
dd

[D(objectRef=0x7f89e9e05848)

In [44]:
mp = mask_g.getProduct()

In [46]:
names = mp.getBandNames()

In [65]:
names[0]

'u10_time2648'

In [3]:
from netCDF4 import Dataset

In [4]:
f = Dataset('../da/Ascending_S1A_IW_GRDH_1SDV_20170723T225031_20170723T225056_017605_01D74D_AA68.nc')

In [8]:
array = f.variables['u10_time2648'][:]

In [5]:
x = np.array([[1,0],[2,4]])

In [12]:
np.mean(array, axis=1)

masked_array(data=[2.8936018766934186, 2.8908938986406665,
                   2.8805291522340815, 2.86475178364388,
                   2.8614361983272145, 2.8767388997887484,
                   2.930993352595494, 2.961274180362563,
                   2.9719518445641855, 2.9586305304361593,
                   2.7831184836250054, 2.870254143891623,
                   2.9387467213360057],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False],
       fill_value=1e+20)

In [13]:
array.fill_value

-32767

In [15]:
 array[0,1] is np.ma.masked

True

In [22]:
x = np.ma.array([[1,2,3],[3,4,5]], mask=[[True, False,True],[True, False,False]])

In [23]:
np.mean(x, axis=1)

masked_array(data=[2.0, 4.5],
             mask=[False, False],
       fill_value=1e+20)